In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.patches
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import pandas as pd
import flopy

In [ ]:
# make modflow model
outflow_boundary_type = "zdg" # cdb or zdg
exe_name = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim_ws = Path(f"./mf6_vcatch_{outflow_boundary_type}")
name = "vcatch"

Lx = 800.0 + 800.0 + 20.0
Ly = 1000.0
dx = 20.0
nrow = int(Ly / dx)
ncol = int(Lx / dx)
nlay = 1

slope_x = 0.05
slope_y = 0.02

x = np.linspace(-Lx / 2 + dx / 2, Lx / 2 - dx / 2, ncol)
y = np.linspace(Ly - dx / 2, dx / 2, nrow)
X, Y = np.meshgrid(x, y)
land_surface = np.abs(X) * slope_x + Y * slope_y

rough_overland = 0.015
rough_channel = 0.15

rainfall = 3.0e-6  # meters per second
time_rainfall = 90  # minutes
dt0 = 5  # seconds
dtmin = 5
dtmax = 100.0  # seconds
dtadj = 2.0
dtfailadj = 5.0

nper = 2
perlen = nper * [
    time_rainfall * 60.0
]  # convert time_rainfall from minutes to seconds
nstp = nper * [1]
tsmult = nper * [1]

tdis_rc = []
for i in range(nper):
    tdis_rc.append((perlen[i], nstp[i], tsmult[i]))

name = "swf"

# build MODFLOW 6 files
sim = flopy.mf6.MFSimulation(
    sim_name=f"{name}_sim",
    version="mf6",
    exe_name=exe_name,
    sim_ws=sim_ws,
)

# create tdis package
ats_filerecord = None
tdis = flopy.mf6.ModflowTdis(
    sim,
    ats_filerecord=ats_filerecord,
    time_units="SECONDS",
    nper=nper,
    perioddata=tdis_rc,
)

# setup ats
ats_filerecord = name + ".ats"
atsperiod = [
    (0, dt0, dtmin, dtmax, dtadj, dtfailadj),
    (1, dt0, dtmin, dtmax, dtadj, dtfailadj),
]
tdis.ats.initialize(
    maxats=len(atsperiod),
    perioddata=atsperiod,
    filename=ats_filerecord,
)

# surface water model
swfname = f"{name}_model"
swf = flopy.mf6.ModflowSwf(sim, modelname=swfname, save_flows=True)

nouter, ninner = 15, 100
hclose, rclose, relax = 1e-8, 1e-8, 1.0
imsswf = flopy.mf6.ModflowIms(
    sim,
    print_option="SUMMARY",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="DBD",
    under_relaxation_theta=0.9,
    under_relaxation_kappa=0.0001,
    under_relaxation_gamma=0.0,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    # backtracking_number=5,
    # backtracking_tolerance=1.0,
    # backtracking_reduction_factor=0.3,
    # backtracking_residual_limit=100.0,
    filename=f"{swfname}.ims",
)
sim.register_ims_package(imsswf, [swf.name])

botm = land_surface.reshape((nlay, nrow, ncol))
dis = flopy.mf6.ModflowSwfdis(
    swf,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=dx,
    delc=dx,
    top=100.0,
    botm=botm,
    xorigin=-810,
)

rough = rough_overland * np.ones((nlay, nrow, ncol), dtype=float)
rough[0, :, int(ncol / 2)] = rough_channel

dfw = flopy.mf6.ModflowSwfdfw(
    swf,
    print_flows=False,
    save_flows=True,
    manningsn=rough,
    idcxs=None,
)

sto = flopy.mf6.ModflowSwfsto(
    swf,
    save_flows=True,
    steady_state={0: False, 1: False},
    transient={0: True, 1: True},
)

ic = flopy.mf6.ModflowSwfic(
    swf,
    strt=botm,
)

# output control
oc = flopy.mf6.ModflowSwfoc(
    swf,
    budget_filerecord=f"{swfname}.bud",
    stage_filerecord=f"{swfname}.stage",
    saverecord=[
        ("STAGE", "ALL"),
        ("BUDGET", "ALL"),
    ],
    printrecord=[
        ("BUDGET", "ALL"),
    ],
)

# flw
qinflow = rainfall * dx * dx
spd = [(0, i, j, qinflow) for j in range(ncol) for i in range(nrow)]
flw = flopy.mf6.ModflowSwfflw(
    swf,
    maxbound=len(spd),
    print_input=True,
    print_flows=True,
    stress_period_data={0: spd, 1: []},
)

if outflow_boundary_type == "zdg":
    fname = f"{swfname}.outflow.csv"
    zdg_obs = {
        fname: [
            ("OUTFLOW", "ZDG", (0, nrow - 1, int(ncol / 2))),
        ],
        "digits": 10,
    }

    idcxs = -1  # use cross section 0
    width = dx
    spd = [
        ((0, nrow - 1, int(ncol / 2)), idcxs, width, slope_y, rough_channel)
    ]
    zdg = flopy.mf6.ModflowSwfzdg(
        swf,
        observations=zdg_obs,
        print_input=True,
        maxbound=len(spd),
        stress_period_data=spd,
    )

elif outflow_boundary_type == "cdb":
    fname = f"{swfname}.outflow.csv"
    cdb_obs = {
        fname: [
            ("OUTFLOW", "CDB", (0, nrow - 1, int(ncol / 2))),
        ],
        "digits": 10,
    }

    idcxs = -1  # use cross section 0
    width = dx
    spd = [
        ((0, nrow - 1, int(ncol / 2)), idcxs, width)
    ]
    cdb = flopy.mf6.ModflowSwfcdb(
        swf,
        observations=cdb_obs,
        print_input=True,
        maxbound=len(spd),
        stress_period_data=spd,
    )

# write the model files
sim.write_simulation()

# run the model
sim.run_simulation()

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
pmv = flopy.plot.PlotMapView(model=swf)
cs = pmv.contour_array(dis.botm.array, colors="k")
ax.clabel(cs, inline=1, fontsize=10)
rect = matplotlib.patches.Rectangle((-10, 0), 20, 1000, facecolor="gray")
ax.add_patch(rect)

plt.xlabel("X location, in meters")
plt.ylabel("Y location, in meters")
fname = Path("../../doc/figures/") / "vcatch-surface.png"
plt.savefig(fname)

In [ ]:
# load background image for comparison
fpth = Path("./data/digiammarco1996.dat")
df_digiammarco = pd.read_csv(fpth, sep=" ", header=None, names=["time", "flow"])
fpth = Path("./data/vanderkwaak1999.dat")
df_vanderkwaak = pd.read_csv(fpth, sep=" ", header=None, names=["time", "flow"])
fpth = Path("./data/panday2004.dat")
df_panday = pd.read_csv(fpth, sep=" ", header=None, names=["time", "flow"])

fpth = sim_ws / f"swf_model.outflow.csv"
obsvals = np.genfromtxt(fpth, names=True, delimiter=",")

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot(df_digiammarco["time"], df_digiammarco["flow"], "r-", label="di Giammarco et al (1996)")
ax.plot(df_vanderkwaak["time"], df_vanderkwaak["flow"], "g-", label="Vanderkwaak (1999)")
ax.plot(df_panday["time"], df_panday["flow"], "b-", label="Panday and Huyakorn (2004)")
ax.plot(
    obsvals["time"] / 60.0,
    -obsvals["OUTFLOW"],
    marker="o",
    markersize=4,
    mfc="none",
    mec="k",
    lw=0.0,
    label=f"MODFLOW 6",
)
ax.plot([90, 90], [0, 5], "k--")
ax.set_xlim(0, 180.0)
ax.set_ylim(0, 5)
ax.set_aspect(20)
plt.xlabel("Time, in minutes")
plt.ylabel("Simulated Outflow,\nin cubic meters per second")
plt.legend(loc="upper right", fontsize=7)
fname = Path("../../doc/figures/") / "vcatch-results.png"
plt.savefig(fname)

In [ ]:
# Compare zdg with cdb
fpth = sim_ws / f"swf_model.outflow.csv"
sim_ws = Path(f"./mf6_vcatch_cdb")
fpth = sim_ws / f"swf_model.outflow.csv"
cdb_obsvals = np.genfromtxt(fpth, names=True, delimiter=",")
sim_ws = Path(f"./mf6_vcatch_zdg")
fpth = sim_ws / f"swf_model.outflow.csv"
zdg_obsvals = np.genfromtxt(fpth, names=True, delimiter=",")

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot(
    cdb_obsvals["time"] / 60.0,
    -cdb_obsvals["OUTFLOW"],
    marker="o",
    markersize=4,
    mfc="none",
    mec="b",
    lw=0.0,
    label=f"CDB",
)
ax.plot(
    zdg_obsvals["time"] / 60.0,
    -zdg_obsvals["OUTFLOW"],
    marker="x",
    markersize=4,
    mfc="none",
    mec="r",
    lw=0.0,
    label=f"ZDG",
)
ax.plot([90, 90], [0, 5], "k--")
ax.set_xlim(0, 180.0)
ax.set_ylim(0, 5)
ax.set_aspect(20)
plt.xlabel("Time, in minutes")
plt.ylabel("Simulated Outflow,\nin cubic meters per second")
plt.legend(loc="upper right", fontsize=7)
#fname = Path("../../doc/figures/") / "vcatch-results.png"
#plt.savefig(fname)